In [2]:
%reload_ext autoreload
%autoreload 0

import os
os.sys.path.append('/home/kirill/Dropbox/tmpworkspace/github/python_recipes')

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from scipy import stats

from utils.misc import parse_int

df = pd.read_csv('adac.csv')
df.describe(include='all')

,2011,2012,2013,2014,2015,2016,2017,2018,kw,mark,model
count,1725.000000,2541.000000,3554.000000,4115.000000,4642.000000,4519.000000,3685.000000,3094.000000,7439.000000,7504,7504
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,5890
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mercedes,"(II)"""
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,760,22
mean,11207.739130,12653.650138,16870.843275,20535.037424,22928.578199,26980.915025,28831.312347,41822.532321,134.708294,NaN,NaN
std,8612.629135,10635.033068,14716.917537,17771.356493,18712.294730,20413.202692,22963.346641,30689.243116,76.889576,NaN,NaN
min,2575.000000,2850.000000,3225.000000,3925.000000,4050.000000,4600.000000,5300.000000,6950.000000,9.000000,NaN,NaN
25%,6850.000000,7550.000000,9150.000000,10750.000000,12550.000000,15300.000000,16000.000000,23750.000000,84.000000,NaN,NaN
50%,8900.000000,9950.000000,12300.000000,14750.000000,17250.000000,21450.000000,22550.000000,33850.000000,110.000000,NaN,NaN
75%,12950.000000,14100.000000,19100.000000,23800.000000,26100.000000,31412.500000,33150.000000,48247.500000,154.000000,NaN,NaN


In [3]:
def is_suv(x):
    return any(
        [v.lower() in x.lower() 
         for v in ['sportage', '3008', 'tiguan', 'compass', 'yeti', 'rav4', 'grandland'
                   'karoq', 'ateca', 'scenic', 'tucson']]
    )

df = df[df['model'].apply(is_suv)]
df.head()

,2011,2012,2013,2014,2015,2016,2017,2018,kw,mark,model
2600,NaN,NaN,NaN,NaN,12800.0,14350.0,16050.0,22840.0,97.0,Hyundai,Tucson 1.6 GDI blue Classic (TL)
2601,NaN,NaN,NaN,NaN,15650.0,17550.0,19650.0,28190.0,130.0,Hyundai,Tucson 1.6 Turbo Trend (TL)
2602,NaN,NaN,NaN,NaN,16750.0,18750.0,21000.0,30090.0,130.0,Hyundai,Tucson 1.6 Turbo Trend Allrad (TL)
2603,NaN,NaN,NaN,NaN,13150.0,14400.0,16150.0,25190.0,85.0,Hyundai,Tucson 1.7 CRDi blue Classic (TL)
2604,NaN,NaN,NaN,NaN,15900.0,17600.0,19750.0,29840.0,100.0,Hyundai,Tucson 2.0 CRDi blue Trend (TL)


In [4]:
years = ['2018', '2017']

for year in years:
    df = df[df[year].notna()]

df = df.filter(years + ['kw', 'mark', 'model'])

df.head()

,2018,2017,kw,mark,model
2600,22840.0,16050.0,97.0,Hyundai,Tucson 1.6 GDI blue Classic (TL)
2601,28190.0,19650.0,130.0,Hyundai,Tucson 1.6 Turbo Trend (TL)
2602,30090.0,21000.0,130.0,Hyundai,Tucson 1.6 Turbo Trend Allrad (TL)
2603,25190.0,16150.0,85.0,Hyundai,Tucson 1.7 CRDi blue Classic (TL)
2604,29840.0,19750.0,100.0,Hyundai,Tucson 2.0 CRDi blue Trend (TL)


In [21]:
data = df[df['mark'] == 'Peugeot']

print(data.size)

print(data.head())

size = data.size
# size = 2


# fig, axs = plt.subplots(ncols=size)
for i in range(size):
    print(data.iloc[i])
    matrix = [
        [int(year), data.iloc[i][year]] 
        for year in years
    ]
    print(matrix)
    row_data = pd.DataFrame(matrix, columns=['year', 'price'])
    print(row_data)
#     sns.scatterplot(x='year', y='price', data=row_data)


    
# print(data.index.tolist())
# d = data.loc[4928, :]
# print(d['mark'])

# x ''
# pal = dict(Lunch="seagreen", Dinner="gray")
# g = sns.FacetGrid(tips, hue="time", palette=pal, height=5)
# g.map(plt.scatter, x, y, s=50, alpha=.7, linewidth=.5, edgecolor="white")
# g.add_legend();



25
         2018     2017     kw     mark                                  model
4928  23650.0  17350.0   96.0  Peugeot      3008 1.2 PureTech 130 Access (II)
4929  32500.0  24250.0  121.0  Peugeot      3008 1.6 THP 165 Allure EAT6 (II)
4930  30300.0  21050.0   96.0  Peugeot       3008 1.5 BlueHDi 130 Active (II)
4931  37850.0  27300.0  130.0  Peugeot  3008 2.0 BlueHDi 180 Allure EAT8 (II)
4932  40050.0  29450.0  133.0  Peugeot      3008 2.0 BlueHDi 180 GT EAT6 (II)
2018                                 23650
2017                                 17350
kw                                      96
mark                               Peugeot
model    3008 1.2 PureTech 130 Access (II)
Name: 4928, dtype: object
[[2018, 23650.0], [2017, 17350.0]]
   year    price
0  2018  23650.0
1  2017  17350.0
2018                                 32500
2017                                 24250
kw                                     121
mark                               Peugeot
model    3008 1.6 THP 165 Allu

IndexError: single positional indexer is out-of-bounds